<a href="https://colab.research.google.com/github/JericCantos/retail_demand_analysis/blob/main/Masterschool_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Data Part 1

In [ ]:
import pandas as pd
import requests
import io
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Build the direct download URL from a file ID
def make_drive_url(file_id):
    return f"https://drive.google.com/uc?id={file_id}"

# Helper function to load a CSV from a direct URL
def load_csv_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # Raises an error if the request fails
    return pd.read_csv(io.StringIO(response.text))

# Dictionary of file IDs for clarity
file_ids = {
    "holiday_events": "1RMjSuqHXHTwAw_PGD5XVjhA3agaAGHDH",
    "items": "1ogMRixVhNY6XOJtIRtkRllyOyzw1nqya",
    "oil": "1Q59vk2v4WQ-Rpc9t2nqHcsZM3QWGFje_",
    "stores": "1Ei0MUXmNhmOcmrlPad8oklnFEDM95cDi",
    "train": "1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv",
    "transactions": "1PW5LnAEAiL43fI5CRDn_h6pgDG5rtBW_"
}

# Load each CSV using the helper functions
df_holiday_events = load_csv_from_url(make_drive_url(file_ids["holiday_events"]))
df_items          = load_csv_from_url(make_drive_url(file_ids["items"]))
df_oil            = load_csv_from_url(make_drive_url(file_ids["oil"]))
df_stores         = load_csv_from_url(make_drive_url(file_ids["stores"]))
# df_train          = load_csv_from_url(make_drive_url(file_ids["train"])) we dont read it as the file is too big and wont work this way
df_transactions   = load_csv_from_url(make_drive_url(file_ids["transactions"]))


In [ ]:
df_items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


# Loading Data Part II - Training Data

Now, all of the files were read in well in the previous lecture except for the `train.csv` file, which contains time-series data for each item sold in a store. Previously, we’ve mentioned that the `train.csv`  file is very large. To read in such big files, we typically split it into chunks and read it in chunk by chunk. Below, you’ll see how we do it but before looking at the actual code it is also important to mention that the train.cvs data file we’ll be filtered down even further by:

1. selecting only data for “Pichincha” region - the region of our analysis
2. selecting only 2 000 000 rows (to make further computations fast for educational sake)

So, let’s do all of these steps now: (to keep things clean, we will reuse the functions and variables we already defined in the previous lecture).

## 1. Install the gdown Library

`gdown` handles large files better than `requests`

In [ ]:
!pip install -U gdown

## 2. Download `train.csv`

In [ ]:
import gdown

# Use our existing function to build the download URL
train_url = make_drive_url(file_ids["train"])

# Download the file using gdown
gdown.download(train_url, "train.csv", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv
From (redirected): https://drive.google.com/uc?id=1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv&confirm=t&uuid=80793780-41c9-4cac-a90b-c16fe5eb7ff0
To: /content/train.csv
100%|██████████| 5.00G/5.00G [01:07<00:00, 73.6MB/s]


'train.csv'

## 3. Select Stores from Pichincha Region

In [ ]:
store_ids = df_stores[df_stores['state'] == 'Pichincha']['store_nbr'].unique()

## 4. Read the Data in Chunks and Filter by Store

read `train.csv` in chunks of 1 million rows and keep only the rows that belong to the selected stores.

In [ ]:
import pandas as pd

chunk_size = 10**6  # 1 million rows at a time
filtered_chunks = []

for chunk in pd.read_csv("train.csv", chunksize=chunk_size):
    chunk_filtered = chunk[chunk['store_nbr'].isin(store_ids)]
    filtered_chunks.append(chunk_filtered)
    del chunk  # Free up memory


/tmp/ipython-input-8-2678381120.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("train.csv", chunksize=chunk_size):


## 5. Combine the Chunks
combine into a single DataFrame

In [ ]:
df_train = pd.concat(filtered_chunks, ignore_index=True)

## 6. Sample Two Million Rows

just to keep things fast and simple for learning.

In [ ]:
df_train = df_train.sample(n=2_000_000).reset_index(drop=True)

## 7. Clean Up
delete the filtered chunks from memory

In [ ]:
del filtered_chunks

## 8. Check Head

In [ ]:
df_train.head()

# EDA

## 1. Checking for Missing Data


### df_train
-  358k rows with null `onpromotioon` exist. No other null values exist.
- given that this is roughly 18% of the data, dropping the rows will not be a good option.
- Question would be whether to drop the column, or impute it.
- Based on the remainder of the dataset, almost 93% of the rows have `onpromotion = False`, so it seems better to replace nulls with False.

In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum()

In [ ]:
null_rows = df_train.isnull().sum().sum()
null_proportion = null_rows*100 / len(df_train)
null_proportion

In [ ]:
df_train['onpromotion'].value_counts(normalize=True)

In [ ]:
df_train['onpromotion'] = df_train['onpromotion'].fillna(False).astype(bool)

In [ ]:
df_train.isnull().sum()

### df_items
No nulls

In [ ]:
df_items.info()

### df_holiday_events
No nulls

In [ ]:
df_holiday_events.info()

### df_oil
- 43 nulls for the `dcoilwtico` column.
- dropping this does not seem to be a good idea since this seems to be a time series too, and each date is unique.
- for now, I will try linear interpolation to fill the nulls.
- since the very first row has a null value, I will just backward fill it.

In [ ]:
df_oil.info()

In [ ]:
df_oil.isnull().sum()

In [ ]:
df_oil['dcoilwtico'].describe()

In [ ]:
len(df_oil['date'].unique()) == len(df_oil)

In [ ]:
df_oil['dcoilwtico'].hist(alpha=0.9, label='dcoilwtico')

plt.xlabel('dcoilwtico')
plt.xticks(np.arange(0, 120, step=10))
plt.ylabel('frequency')
plt.show()

In [ ]:
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].interpolate(method='linear')
df_oil.isnull().sum()

In [ ]:
df_oil.loc[df_oil['dcoilwtico'].isnull()]

In [ ]:
#backward fill the first row
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].bfill()
df_oil.isnull().sum()

In [ ]:
df_oil.iloc[:2, :]

### df_stores
No nulls

In [ ]:
df_stores.info()

### df_transactions
No nulls

In [ ]:
df_transactions.info()

## Handle Outliers

In [ ]:
df_train.describe()

### Treat product returns as `no sale`

In [ ]:
#replace all negative unit_sales with 0
df_train['unit_sales'] = df_train['unit_sales'].apply(
                        lambda x: 0 if x<0 else x)

df_train.describe()

### Identify extremely high sales for specific stores and items on specific days

- any day where sales were **more than 5x the group's own standard deviation** above its average is an outlier.
- Z-score of 5 represents roughly top 0.00003 % in a normal distribution
- rows with a Z-score > 5 might be promo spikes, data-entry errors, or stock-clearance days.

In [ ]:
def calculate_store_item_zscore(group):
    # Compute mean and standard deviation for each store-item group
    mean_sales = group['unit_sales'].mean()
    std_sales = group['unit_sales'].std()

    # Calculate Z-score for unit_sales
    # (avoiding division by zero for standard deviation),
    # and store it in a new column called z_score
    group['z_score'] = (group['unit_sales'] - mean_sales) / (std_sales
                                                             if std_sales != 0
                                                             else 1)
    return group

In [ ]:
# Apply the Z-score calculation to each store-item group, then flatten the index
df_train_grouped = df_train.groupby(['store_nbr', 'item_nbr'])\
                            .apply(calculate_store_item_zscore,
                                   include_groups=True)
df_train_grouped.reset_index(drop=True, inplace=True)

# Define threshold for outliers (e.g., Z-score > 5)
outliers = df_train_grouped[df_train_grouped['z_score'] > 5]

# Print summary
print(f"Number of outliers detected: {len(outliers)}")
outliers.head()

### Things to consider:
- were these during promotions i.e. `onpromotion = True`?
- Is the data a holiday?
- Could it be a data glitch? (e.g., negative sales on the day before followed by a big positive “correction”)

--- personal experimentation below ---

In [ ]:
df_holiday_events.loc[(df_holiday_events['transferred']==True)].head()

In [ ]:
df_holiday_events.sort_values('date').head()

In [ ]:
#merge outliers with holidays

outliers_copy = pd.merge(outliers, df_holiday_events, how='left', on='date')

outliers_copy.head()

In [ ]:
not_holiday = len(outliers_copy[outliers_copy['type'].isnull()])
on_holiday = len(outliers_copy[~outliers_copy['type'].isnull()])
total = len(outliers_copy)

print('Total Outliers:', total, ', On Holiday:', on_holiday,
      ', Not on Holiday', not_holiday)

In [ ]:
on_promotion = len(outliers_copy[outliers_copy['onpromotion']==True])
not_promotion = len(outliers_copy[outliers_copy['onpromotion']==False])
total = len(outliers_copy)

print('Total Outliers:', total, ', On Promotion:', on_promotion,
      ', Not on Promotion', not_promotion)

In [ ]:
# check what the union of holidays and promotions are

on_promotion_or_holiday = len(outliers_copy[(outliers_copy['onpromotion']==True)
                              | (~outliers_copy['type'].isnull()) ])

not_promotion_nor_holiday = len(outliers_copy[(outliers_copy['onpromotion']==False)
                              & (outliers_copy['type'].isnull()) ])

total = len(outliers_copy)

print('Total Outliers:', total,
      '\n On Promotion or Holiday:', on_promotion_or_holiday,
      '\n Not on Promotion nor Holiday', not_promotion_nor_holiday,
      '\n % on Promotion nor Holiday', on_promotion_or_holiday*100/total)

\~55% of the outliers could be related to either a promotion or a holiday. A significant chunk would still be unexplained (\~45%).

## Fill Missing Dates with Zero Sales
- because this is a time series, we expect a **complete calendar**.
- we should fill the dates where there are zero sales with explicit zeros so that there are no gaps in our calendar.

**Goal**: every product in every store has one row per calendar day.
- If nothing sold, `unit_sales` should be `0`

In [ ]:
# Turn the date column into real dates

df_train['date'] = pd.to_datetime(df_train['date'])

In [ ]:
# define a function to create a full daily calendar
# for every store-item pair
# expect df_train.groupby(['store_nbr', 'item_nbr'])

def fill_calendar(group):
  #
  # group contains all rows for ONE (store_nbr, item_nbr) pair
  #
  g = group.set_index("date").sort_index()   # use date/calendar as the index

  g = g.asfreq("D", fill_value=0)
  # make it daily; add 0 to all columns where the date is missing

	# put the identifiers back (asfreq drops them)
  g["store_nbr"] = group["store_nbr"].iloc[0]
  g["item_nbr"] = group["item_nbr"].iloc[0]

  return g.reset_index()                     # date back to a normal column


In [ ]:
# Apply the function to every store-item pair
df_train = (
    df_train
    .groupby(["store_nbr", "item_nbr"], group_keys=False)  # keeps memory low
    .apply(fill_calendar)
)

df_train.head()

/tmp/ipython-input-44-2527799502.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(fill_calendar)


,date,id,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-02-05,1384585,1,96995,1.0,False
1,2013-02-06,0,1,96995,0.0,0
2,2013-02-07,0,1,96995,0.0,0
3,2013-02-08,0,1,96995,0.0,0
4,2013-02-09,0,1,96995,0.0,0


In [ ]:
from datetime import datetime
df_2017 = df_train[df_train['date'] >=  datetime.strptime('2017-01-01', '%Y-%m-%d')]
df_2017.head()

,date,id,store_nbr,item_nbr,unit_sales,onpromotion
1426,2017-01-01,0,1,96995,0.0,0
1427,2017-01-02,0,1,96995,0.0,0
1428,2017-01-03,0,1,96995,0.0,0
1429,2017-01-04,0,1,96995,0.0,0
1430,2017-01-05,0,1,96995,0.0,0


Result:
- `df_train` has every day for every product in every store. i.e. `df_train` now contains every calendar day for every (store_nbr, item_nbr).

- Missing days in the original data are present with unit_sales = 0 meaning nothing was sold.
The DataFrame has a fresh 0…N index;

## Feature Engineering




### Turn a date into useful signals

| New Feature | Why it helps the model |
| ----------- | ---------------------- |
| `year` | Captures long-term growth or decline, e.g. sales rise every year |
| `month` | Picks up holiday seasons (November - December), back-to-school spikes, etc. |
| `day` | Useful for month-end or mid-month payday surges. |
| `day_of_week` | Reveals weekend vs. weekday patterns - Friday grocery rush, Sunday lull |

Add these to the dataset.

With these columns in place, even a simple tree-based model can learn that “sales usually jump in December, dip on Mondays, and peak on the last day of each month.”  That extra context often boosts forecast accuracy without complex algorithms.

In [ ]:
# Ensure 'date' is a real datetime
df_train['date'] = pd.to_datetime(df_train['date'])

# Split the timestamp
df_train['year'] = df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month
df_train['day'] = df_train['date'].dt.day
df_train['day_of_week'] = df_train['date'].dt.day_of_week
# Monday=0 … Sunday=6

df_train.head()

### Rolling / Moving Averages
7-day rolling mean for `unit_sales`

In [ ]:
# 7-day rolling average of unit_sales, per (item, store)
df_train = df_train.sort_values(["item_nbr", "store_nbr", "date"]).reset_index(drop=True) # make sure rows are in time order

df_train["unit_sales_7d_avg"] = (
  df_train
  .groupby(["item_nbr", "store_nbr"])["unit_sales"]
    # isolate one time-series per (item, store), get the units sold
  .transform(lambda s: s.rolling(window=7, min_periods=1).mean())
    #  mean of last 7 days, i.e. 7-day moving average, aligned back to original df
)

In [ ]:
# Lets see how the new column unit_sales_7d_avg looks like.
#For that, we'll need to select a store and item.
# Get store and item from the first row
store_id = df_train.iloc[0]['store_nbr']
item_id = df_train.iloc[0]['item_nbr']

# Can also explore other store-item pairs
random_row = df_train.sample(1).iloc[0]
store_id = random_row['store_nbr']
item_id = random_row['item_nbr']

# Filter the DataFrame for this store-item pair
sample = df_train[(df_train['store_nbr'] == store_id) & (df_train['item_nbr'] == item_id)]
sample.head()

## Visualizing Time Series Data

### a) Sales Over Time (Aggregated)

- high level view of how sales changed over time.
- overall sales trend for all stores and items
  - group data by date to get one row per day
  - sum up `unit_sales` on each day across all stores and products

In [ ]:
import matplotlib.pyplot as plt


sales_by_date = df_train.groupby('date')['unit_sales'].sum()

# Plotting the time-series
plt.figure(figsize=(12,6))
plt.plot(sales_by_date.index, sales_by_date.values)
plt.title('Total Unit Sales Over Time in Pichincha state', fontsize=20, fontweight='bold')
plt.xlabel('Date', fontsize=16)
plt.ylabel('Unit Sales', fontsize=16)
plt.xticks(fontsize=14, rotation=45)
plt.yticks(fontsize=14)
plt.show()

### b) Sales Trend by Year and Month
- find seasonality / seasonal patterns
- aggregate by year and month

In [ ]:
# unstack reshapes the results so that:
# - each row is a year
# - each column is a month (1-12)
# - the values are the total sales
sales_by_month = (
    df_train.groupby(['year', 'month'])['unit_sales'].sum().unstack()
)

In [ ]:
# create a heatmap

import seaborn as sns

plt.figure(figsize=(8,5))

sns.heatmap(
    sales_by_month,
    cmap='coolwarm',  # Use a diverging colormap for better contrast
    linewidths=0.5,  # Add lines between cells for clarity
    linecolor='white',  # Use white lines for a cleaner look
    cbar_kws={'label': 'Sales Volume'}  # Add a descriptive colorbar label
)

# Customizing title and axes labels
plt.title('Monthly Sales Trends Over Years', fontsize=22, fontweight='bold')
plt.xlabel('Month', fontsize=18, labelpad=10)  # Labelpad adds spacing
plt.ylabel('Year', fontsize=18, labelpad=10)

# Formatting tick labels
plt.xticks(fontsize=14, rotation=45)  # Rotate x-axis labels for better readability
plt.yticks(fontsize=14)

# Adjust layout for better spacing
plt.tight_layout()

# Display the heatmap
plt.show()

## Examining the Impact of Holidays

- Holidays have a big impact in retail
- link daily_sales from `df_train` with the holidays in `df_holiday_events`
- see how much `unit_sales` change when a day is flagged as a holiday

In [ ]:
df_holiday_events.head()

In [ ]:
df_holiday_events['type'].unique()

In [ ]:
# cover `date` column to a real datetime and check range
df_holiday_events['date'] = pd.to_datetime(df_holiday_events['date'])

print(
    'Holiday file covers:',
    df_holiday_events['date'].dt.date.min(),
    df_holiday_events['date'].dt.date.max()
)

In [ ]:
# join holidays into sales table
df_train_holiday = pd.merge(
    df_train,                     # daily sales
    df_holiday_events[['date', 'type']],  # keep only what we need
    on='date',
    how='left'                    # non-holiday days get NaN in 'type'
)
df_train_holiday.head()

### Compare average sales for each holiday type
"On an average dayy, how many units sell when it's a Holiday vs. a normal Work Day"?

- group by `type`
- take mean of `unit_sales`
- plot in a bar chart

In [ ]:
holiday_sales = df_train_holiday.groupby('type')['unit_sales'].mean()

holiday_sales.plot(
    kind='bar',
    figsize=(8,5),
    color='lightgreen',
    edgecolor = 'black'
)

plt.title('Average Unit Sales by Day Type', fontsize=18, weight='bold')
plt.ylabel('Average units sold')
plt.xticks(rotation=0)
plt.show()

## Analyzing Perishable Items

- perishable items are goods with a limited shelf life or spoil quickly.
- forecasting demand for perishables is **business critical**
  - over-order, we take a loss from expired items
  - under-order, miss sales and disappoint customers

1. add the `perishable` flag from `df_items` to `df_train`
2. set the `perishable` column to type `boolean`
3. compare total sales for perishable vs. non-perishable

In [ ]:
df_items.head()

In [ ]:
# join the tables
# set perishable to boolean
df_train_items = pd.merge (
    df_train,
    df_items,
    on='item_nbr',
    how='left'
)

df_train_items['perishable'] = df_train_items['perishable'].astype(bool)
df_train_items.head()

In [ ]:
# group sales by perishable
perishable_sales = df_train_items.groupby('perishable')['unit_sales'].sum()

# plot bar chart
perishable_sales.plot(
    kind='bar',
    figsize=(12, 6),
    color=['orange', 'green'],
    edgecolor='black'
)
plt.title('Sales of Perishable vs Non-Perishable Items', fontsize=16)
plt.ylabel('Total Unit Sales', fontsize=16)
plt.xlabel('')
plt.xticks(
    ticks=[0, 1],
    labels=['Non-Perishable', 'Perishable'],
    fontsize=16,
    rotation=0  # Keep x-axis labels horizontal
)
plt.yticks(fontsize=14)
plt.show()

## Analyzing the Impact of Oil Prices

Exercise: Do oil prices move with our sales?

Objective: investigate whether daily crude-oil prices have any visible relationship with daily unit sales

Given:
- `df_train` - cleained daily sailes (`date`, `unit_sales`, etc.)
- `df_oil` - daily WTI oil prices (`date`, `dcoilwtico`)

Expected Output:
1. plot that lets you see both time series together
2. short note: any obvious relationship?

Impressions:
- The oil prices in 2013 were relatively high, correlating with the lower overall sales.
- the oil pricesdropped in the latter part of 2014, wwhich coincided with a spike in purchasing.
- lower oil prices seem to show higher sales.

In [ ]:
# Make sure the date column is a real datetime
df_oil['date'] = pd.to_datetime(df_oil['date'])

daily_sales = df_train.groupby('date')['unit_sales'].sum()

# Merging df_train with oil data on date
df_train_oil = pd.merge(daily_sales, df_oil, on='date', how='left')

# Plotting oil price vs unit sales
fig, ax1 = plt.subplots(figsize=(10,6))

ax1.set_xlabel('Date')
ax1.set_ylabel('Oil Price', color='tab:blue')
ax1.plot(df_train_oil['date'], df_train_oil['dcoilwtico'], color='tab:blue', label='Oil Price')
ax2 = ax1.twinx()
ax2.set_ylabel('Unit Sales', color='tab:green')
ax2.plot(df_train_oil['date'], df_train_oil['unit_sales'], color='tab:green', label='Unit Sales')

plt.title('Oil Price vs Unit Sales Over Time', fontsize=16)
plt.show()

## Autocorrelation

In [ ]:
from pandas.plotting import autocorrelation_plot

# Aggregate total sales per day
sales_by_date = df_train.groupby('date')['unit_sales'].sum()

# Plot autocorrelation
plt.figure(figsize=(10, 5))
autocorrelation_plot(sales_by_date)
plt.title('Autocorrelation of Daily Unit Sales', fontsize=16)
plt.show()


## Stationarity

In [ ]:
sales_by_date.plot(figsize=(12,5), title='Total Sales Over Time')
plt.ylabel('Unit Sales')
plt.show()


In [ ]:
rolling_mean = sales_by_date.rolling(window=12).mean()
rolling_std = sales_by_date.rolling(window=12).std()

plt.figure(figsize=(12,5))
plt.plot(sales_by_date, label='Original')
plt.plot(rolling_mean, label='Rolling Mean', color='red')
plt.plot(rolling_std, label='Rolling Std', color='green')
plt.title('Rolling Mean & Standard Deviation')
plt.legend()
plt.show()

### ADF

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(sales_by_date)
print("ADF Statistic:", result[0])
print("p-value:", result[1])

### Diagnosing Trend and Seasonality


In [ ]:
# STL decomposition
stl = STL(sales_by_date, period=7)  # again, adjust period based on your seasonality
res = stl.fit()

# Plot STL decomposition
res.plot()
plt.suptitle("STL Decomposition", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate strength of trend and seasonality
# Based on Hyndman’s definition: Strength = 1 - (variance of remainder / variance of (component + remainder))

import numpy as np

trend_strength = 1 - (np.var(res.resid) / np.var(res.trend + res.resid))
seasonal_strength = 1 - (np.var(res.resid) / np.var(res.seasonal + res.resid))

print(f"Strength of Trend: {trend_strength:.2f}")
print(f"Strength of Seasonality: {seasonal_strength:.2f}")